In [14]:
import pandas as pd
import numpy as np

### 1. Import data

In [15]:
# maybe use np.where to reduce the data imported

Read

In [16]:
code = pd.read_csv('data/coding7128.tsv', sep='\t')

In [17]:
clinical = pd.read_csv('data/tpp_gp_clinical_subset.csv')

In [18]:
ic10 = pd.read_csv('data/ic10_lookup.csv')

In [19]:
GPic10 = pd.read_csv('data/GP_ic10_lookup.csv')

In [20]:
scripts = pd.read_csv('data/sample_covid19_tpp_gp_scripts.txt')

In [21]:
code

,coding,meaning
0,-1,redacted - potentially sensitive or identifying
1,-2,redacted - rare occupation
2,.....,Read thesaurus
3,.0...,See 0....
4,.01..,See 01...
...,...,...
265731,yz4U.,2015-12-01 THANATOS
265732,yz4V.,2016-01-01 GRAMMEZA
265733,yz4W.,2016-02-01 MARMARAS
265734,yz4X.,2016-03-01 MATHRAKI


In [22]:
clinical

,Unnamed: 0,eid,event_dt,code_type,code,value
0,101084316,4392480,31/12/2020,0,XaERu,11.00
1,9359934,1311969,31/12/2020,0,XaN3z,0.80
2,9360007,1311969,31/12/2020,0,44P6.,0.58
3,111269383,4731119,31/12/2020,0,XaIm8,0.00
4,9360002,1311969,31/12/2020,0,44P5.,1.30
...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00
49998,116371391,4900832,31/12/2019,0,X1004,0.00


In [23]:
ic10

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
GPic10

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
0,0,123..,Z831,D,C,M,0.0,0.0
1,1,123..,Z830,A,C,M,0.0,0.0
2,2,1231.,Z831,G,C,M,0.0,0.0
3,3,1232.,Z831,G,C,M,0.0,0.0
4,4,1233.,Z831,G,C,M,0.0,0.0
...,...,...,...,...,...,...,...,...
116372,116372,ZVy..,Z769,D,C,C,0.0,0.0
116373,116373,ZVz..,Z769,D,C,C,0.0,0.0
116374,116374,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116375,116375,"Contains information from NHS Digital, licence...",NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
scripts # needs to be complete if want to compare vaccinated vs. non-vaccinated, but seems like not all vaccinations registered here but just the ones done by the GP

,Unnamed: 0,eid,issue_date,dmd_code
0,39278003,3743492,2021-03-22,320031002
1,35914327,3508920,2021-03-15,3591211000001105
2,49790799,4472075,2021-05-03,33405511000001102
3,31447155,3192040,2021-03-15,321745007
4,69964306,5879124,2021-01-19,320031002
...,...,...,...,...
99995,62996879,5393814,2021-01-27,320030001
99996,47834233,4333940,2021-01-03,38893411000001105
99997,33831040,3365447,2021-03-31,318052005
99998,12832421,1889162,2021-03-30,321152004


# finding comorbidities

initial trials

In [ ]:
# mask for getting codes related to myocarditis
mask_myocarditis = code['meaning'].str.contains('myocarditis', case = False)
code[mask_myocarditis]

,coding,meaning
75314,A32y1,Diphtheritic myocarditis
75808,A7423,Coxsackie myocarditis
76092,A93y1,Syphilitic myocarditis
76517,AD03.,Toxoplasma myocarditis
86230,G012.,Acute rheumatic myocarditis
86282,G1y0.,Rheumatic myocarditis
86513,G52..,Acute myocarditis
86514,G520.,Acute myocarditis in diseases EC
86515,G5200,Acute aseptic myocarditis of the newborn
86516,G5201,Acute myocarditis - coxsackie


In [ ]:
code.loc[180816, 'meaning'] # super long meaning, may leave out

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# getting data with myocarditis
myocarditis = code[mask_myocarditis]['coding']
clinical[clinical['code'].isin(myocarditis)]

,Unnamed: 0,eid,event_dt,code_type,code,value


In [ ]:
# mask for getting codes related to heart (also contains heart burn etc!)
mask_heart = code['meaning'].str.contains('heart', case = False)
code[mask_heart]

,coding,meaning
31726,1226.,No FH: Ischaemic heart disease (& [angina])
31854,12C2.,FH: Ischaemic heart disease <60 (& [myocardial...
31855,12C3.,FH: Ischaemic heart disease >60 (& [myocardial...
31857,12C5.,FH: ischaemic heart disease (& [myocardial inf...
31860,12C8.,FH: Congenital heart disease
...,...,...
183078,bm...,Vasodilators in heart failure [no drugs here]
183971,bxdG.,Zocor Heart-Pro 10mg tablet
257251,x02FI,Heart and cardiovascular imaging agents and kits
265054,x05uG,Galpharm Heartburn Relief


In [ ]:
# getting data related to "heart", attention: also contains heartburn! just to check if it works
heart = code[mask_heart]['coding']
clinical[clinical['code'].isin(heart)]['code'].value_counts()

24B1.    7
XE2uV    4
XE0o3    4
XE0oH    3
XE0oG    3
79...    1
5853z    1
X773r    1
G58..    1
Name: code, dtype: int64

In [ ]:
## ishaemic heart disease
mask_ihd = code['meaning'].str.contains('ischaemic heart disease', case = False)
ischaemicHD = code[mask_ihd]['coding']
clinical[clinical['code'].isin(ischaemicHD)]

,Unnamed: 0,eid,event_dt,code_type,code,value
3603,83141410,3791841,31/12/2020,0,XE2uV,0.0
5077,46291289,2557400,31/12/2020,0,XE0oG,0.0
13147,110210389,4695920,31/12/2020,0,XE2uV,0.0
15874,25621154,1854201,31/12/2020,0,XE2uV,0.0
19336,77948891,3619074,31/12/2020,0,XE0o3,0.0
22157,119256026,4996738,31/12/2020,0,XE0o3,0.0
33646,40822178,2372522,31/12/2019,0,XE0o3,0.0
34237,21793392,1726341,31/12/2019,0,XE0oG,0.0
36998,37267699,2253583,31/12/2019,0,XE0oG,0.0
38347,7956420,1265845,31/12/2019,0,XE0oH,0.0


several comorbidities

In [ ]:
# code layout
clinical['disease category'] = np.nan # new column for filling with disease categories
diseases_selec = ['ischaemic heart disease', 'cardiomyopathy', 'heart failure',\
                  'cardiomyopathy'] # list of diseases, to be extended

for disease in diseases_selec:
  mask = code['meaning'].str.contains(disease, case = False) # broadcasting for disease category
  sub_disease = clinical[clinical['code'].isin(code[mask]['coding'])] # subset of data with the disease mask
  for i in sub_disease.index:
    clinical.loc[i,'disease category'] = disease # fill dataset with disease category label

In [ ]:
clinical['disease category'].value_counts()

ischaemic heart disease    14
heart failure               1
Name: disease category, dtype: int64

# finding vaccine

In [ ]:
# # not working like this, because is wrong dataset
# # covid vaccine
# mask_vac = code['meaning'].str.contains('vaccin', case = False)
# mask_cor = code['meaning'].str.contains('cov', case = False)
# mask_corvac = (mask_cor) & (mask_vac)
# code[mask_corvac]
# # clinical[clinical['code'].isin(code[mask_vac]['coding'])]
# # code['meaning'].str.contains('vaccination' and 'corona', case = False)

,coding,meaning
180816,XM1W1,Morning after pill\nXM1W2\tMultip.- A/N data\n...


In [ ]:
# SNOMED code from https://services.nhsbsa.nhs.uk/dmd-browser/amp/view/159421?ref=YW1wTmFtZT1jb3ZpZCZzZWFyY2hUeXBlPUFNUCZzaG93SW52YWxpZEl0ZW1zPWZhbHNl#ampCodes
mask_AZ = scripts['dmd_code']== 39114911000001105
scripts[mask_AZ]

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105
9967,4432489,1304258,2021-02-02,39114911000001105
21275,23019270,2608259,2021-01-22,39114911000001105
23754,30318448,3113257,2021-03-25,39114911000001105
29233,51195703,4568234,2021-01-30,39114911000001105
35668,2884143,1199550,2021-01-21,39114911000001105
37824,61012003,5251075,2021-01-30,39114911000001105


In [ ]:
dict_vac = {39326911000001101: 'Moderna', 39230211000001104: 'Janssen', \
            39826711000001101:'Medicago', 39473011000001103: 'Baxter', \
            39114911000001105: 'AstraZeneca', 39115611000001103: 'Pfizer',\
            39373511000001104: 'Valneva'}
mask_PF = scripts['dmd_code']== 39114911000001105
scripts[mask_PF]

,Unnamed: 0,eid,issue_date,dmd_code
4561,64814430,5521041,2021-01-20,39114911000001105
5499,16646535,2163004,2021-01-31,39114911000001105
7750,13225838,1916265,2021-02-03,39114911000001105
8932,70996867,5949279,2021-02-22,39114911000001105
9967,4432489,1304258,2021-02-02,39114911000001105
21275,23019270,2608259,2021-01-22,39114911000001105
23754,30318448,3113257,2021-03-25,39114911000001105
29233,51195703,4568234,2021-01-30,39114911000001105
35668,2884143,1199550,2021-01-21,39114911000001105
37824,61012003,5251075,2021-01-30,39114911000001105


In [ ]:
for code in dict_vac:
  mask_vac = scripts['dmd_code']== code
  print(dict_vac[code], ':',len(scripts[mask_vac]))

Moderna : 0
Janssen : 0
Medicago : 0
Baxter : 0
AstraZeneca : 23
Pfizer : 7
Valneva : 0


In [ ]:
# get few individuals from large dataset -> may try to call only part of scripts dataset, where code == one of covid vaccine
print('lines in samplescripts: ', len(scripts))
print('number of subjects in samplescripts: ', len(scripts['eid'].value_counts()))

lines in samplescripts:  100000
number of subjects in samplescripts:  51104


# get comorbidities of vaccinated ones

In [ ]:
id_vac = scripts[mask_PF]['eid']

In [ ]:
# finding events of the vaccinated people, but only events before vaccination found -> may need a more current dataset
mask_vac = clinical['eid'].isin(id_vac)
clinical[mask_vac]

,Unnamed: 0,eid,event_dt,code_type,code,value
31073,34474547,2163004,31/12/2020,0,8HE2.,0.000000
36034,47865366,2608259,31/12/2019,0,XaIAQ,36.099998
36086,47865541,2608259,31/12/2019,0,242..,74.000000
36117,47864684,2608259,31/12/2019,0,XaJK8,0.000000
36118,47864707,2608259,31/12/2019,0,XaJKJ,0.000000
36121,47864718,2608259,31/12/2019,0,XaMiI,0.000000
36122,47864755,2608259,31/12/2019,0,XaMhk,0.000000
36126,47864891,2608259,31/12/2019,0,XaJKn,0.000000
36154,47864323,2608259,31/12/2019,0,XE0qn,0.000000
36167,47864473,2608259,31/12/2019,1,Y0529,0.000000


# IC10 codes

https://www.icd10data.com/ICD10CM/Codes
for easy lookup, divided into circulatory etc.

In [ ]:
ic10

,Unnamed: 0,ICD10_CODE,ALT_CODE,USAGE,USAGE_UK,DESCRIPTION,MODIFIER_4,MODIFIER_5,QUALIFIERS,GENDER_MASK,MIN_AGE,MAX_AGE,TREE_DESCRIPTION
0,0,A00,A00,DEFAULT,3.0,Cholera,,,,,,,
1,1,A00.0,A000,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar chol...",,,,,,,
2,2,A00.1,A001,DEFAULT,3.0,"Cholera due to Vibrio cholerae 01, biovar eltor",,,,,,,
3,3,A00.9,A009,DEFAULT,3.0,"Cholera, unspecified",,,,,,,
4,4,A01,A01,DEFAULT,3.0,Typhoid and paratyphoid fevers,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17931,17931,U84.8,U848,DEFAULT,3.0,Resistance to other specified antimicrobial drug,,,,,,,
17932,17932,U84.9,U849,DEFAULT,3.0,Resistance to unspecified antimicrobial drugs,,,,,,,
17933,17933,U85,U85X,DEFAULT,3.0,Resistance to antineoplastic drugs,,,,,,,
17934,17934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


the GP_ic10 has the read code which is the CTV3. However, and when things get nasty, there are multiple ICD10 codes per CTV3 codes and viceversa… there is a mapping status which tells you whether the code is primary, duplicated, to revision, etc

In [ ]:
GPic10['read_code'].unique()


array(['123..', '1231.', '1232.', ..., nan,
       'Contains information from NHS Digital, licenced under the current version of the Open Government Licence available at www.nationalarchives.gov.uk/doc/open-government-licence/open-government-licence.htm',
       'ICD-10 codes, terms and text used by permission of WHO, from: International Statistical Classification of Diseases and Related Health Problems, Tenth Revision (ICD-10). Vols 1-3. Geneva, World Health Organization, 1992-2016.'],
      dtype=object)

In [ ]:
# just to see if corresponding CTV3 to IC10 works
mask_vac_ic10 = GPic10['read_code'].isin(clinical[mask_vac]['code']) # broadcasting for vaccinated comorbidities(CTV3) in ICD10
GPic10[mask_vac_ic10]

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
76606,76606,XE0qn,R05X,D,C,C,0.0,0.0
76607,76607,XE0qn,F453,R,C,C,0.0,0.0
76608,76608,XE0qn,J111,R,C,C,0.0,0.0
76609,76609,XE0qn,J410,R,C,C,0.0,0.0
76610,76610,XE0qn,R042,R,C,C,0.0,0.0


# code layout for getting categories of people w. different comorbidities

In [ ]:
# code layout for getting the different categories

# subsetting into categories using IC10
# e.g. I00-I99 Diseases of the circulatory system, more details available too, e.g. Ischemic heart diseases
GPic10.dropna(inplace=True) # drop NaN to enable making mask

m_circ = GPic10['icd10_code'].str.startswith('I')
GPic10[m_circ]['read_code']

,Unnamed: 0,read_code,icd10_code,mapping_status,refine_flag,add_code_flag,element_num,block_num
302,302,14A3.,I252,G,C,C,0.0,0.0
303,303,14A4.,I252,G,C,C,0.0,0.0
600,600,182..,I209,R,C,C,0.0,0.0
602,602,1822.,I20,R,M,C,0.0,0.0
610,610,1828.,I20,R,M,C,0.0,0.0
...,...,...,...,...,...,...,...,...
115198,115198,Xafis,I480,A,C,C,0.0,0.0
115199,115199,Xafis,I481,A,C,C,0.0,0.0
115200,115200,Xafis,I482,A,C,C,0.0,0.0
115204,115204,Xafyq,I708D,D,M,C,0.0,0.0


In [ ]:
# getting correspondign CTV3 codes
m_circ_CTV3 = code['coding'].isin(GPic10[m_circ]['read_code'])
code[m_circ_CTV3]

,coding,meaning
34065,14A3.,H/O: myocardial infarct at less than 60
34066,14A4.,H/O: myocardial infarct at greater than 60
34881,182..,Chest pain
34883,1822.,Central chest pain
34889,1828.,Atypical chest pain
...,...,...
180522,XM1Qq,Thrombophlebitis
180523,XM1Qr,Superficial vein thrombosis
180524,XM1Qs,Perineal varices
180526,XM1Qu,Claudication


In [ ]:
clinical


,Unnamed: 0,eid,event_dt,code_type,code,value
0,101084316,4392480,31/12/2020,0,XaERu,11.00
1,9359934,1311969,31/12/2020,0,XaN3z,0.80
2,9360007,1311969,31/12/2020,0,44P6.,0.58
3,111269383,4731119,31/12/2020,0,XaIm8,0.00
4,9360002,1311969,31/12/2020,0,44P5.,1.30
...,...,...,...,...,...,...
49995,46497795,2564395,31/12/2019,0,XaFqt,0.00
49996,46497794,2564395,31/12/2019,0,XaIVg,0.00
49997,41639202,2400982,31/12/2019,0,XaEJK,0.00
49998,116371391,4900832,31/12/2019,0,X1004,0.00


In [ ]:
# getting subjects with this condition
m_circ_sub = clinical['code'].isin(code[m_circ_CTV3]['coding']) # another mask
clinical[m_circ_sub]

,Unnamed: 0,eid,event_dt,code_type,code,value
184,30252894,2016840,31/12/2020,0,XE0qq,0.0
285,90244183,4031827,31/12/2020,0,XE0Ub,0.0
543,142831689,5779610,31/12/2020,0,663N1,0.0
964,9359028,1311969,31/12/2020,0,XE0Ub,0.0
1398,92583431,4109720,31/12/2020,0,X79pz,0.0
...,...,...,...,...,...,...
47967,18435643,1614128,31/12/2019,0,H33..,0.0
48260,120887789,5050765,31/12/2019,0,X205b,0.0
48710,26078109,1870891,31/12/2019,0,182..,0.0
49655,32178098,2083247,31/12/2019,0,X75u4,0.0


# correspond to vaccinated or not

In [ ]:
scripts[mask_vac]

,Unnamed: 0,eid,issue_date,dmd_code


In [ ]:
# tbd: define timing (before or after vaccine?)

# no one w. pfizer had circulatory issue before vaccine on this limited dataset
scripts[mask_PF]['eid'].isin(clinical[m_circ_sub]['eid']).sum()

0